### *This Code is Ran in Google Colab*

Hence, You might need to install below libraries in your local machine.

In [4]:
%%capture --no-stderr
%pip install --quiet -U langgraph

In [5]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.9 MB/s eta 0:00:00


##**Simple Graph Overview**

Here, we are trying to create simple graph with below flow -

###*User Input* *--->* *Function 1* *----Edge--->* *Function 2* *--->* *Graph Output*


In [6]:
def function1(input1):
  return input1 + " First Function"

def function2(input2):
  return input2 + " to Second Function"

In [7]:
from langgraph.graph import Graph, StateGraph , START, END

In [8]:
builder = Graph()

builder.add_node("node1", function1)
builder.add_node("node2", function2)

builder.add_edge( "node1", "node2")
builder.set_entry_point("node1")
builder.set_finish_point("node2")

app = builder.compile()

# View
#display(Image(graph.get_graph().draw_mermaid_png()))

In [9]:
app.invoke("I am Travelling from")

'I am Travelling from First Function to Second Function'

In [10]:
app.stream(input="I am Travelling from")

<generator object Pregel.stream at 0x7f3de2990ba0>

In [11]:
# Now if you want to check what output came from each Node our Graph traversed, we can do following -

input = "I am Travelling from"

for output in app.stream(input=input):
  print(output)

{'node1': 'I am Travelling from First Function'}
{'node2': 'I am Travelling from First Function to Second Function'}


## ***Now Trying to Integrate LLM***

In [12]:
import os
from google.colab import userdata

In [13]:
secret_name = "OPENAI_API_KEY"  # Replace with the name you gave your secret
secret_value = userdata.get(secret_name) # Here I am getting the values of API Key from "Colab Secrets"

# If you are using a seperate .env file then you can use dotenv.load_dotenv to load all your variables stored in file say .env file

In [14]:
from langchain_openai import ChatOpenAI

In [15]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo" ,api_key = secret_value,  temperature=0)

In [16]:
llm.invoke("What is the capital of USA?")

AIMessage(content='The capital of the United States of America is Washington, D.C.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 14, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-78a833cd-9aa5-4888-86c7-305cae089346-0', usage_metadata={'input_tokens': 14, 'output_tokens': 14, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
llm.invoke("Hi").content

'Hello! How can I assist you today?'

#### *Now lets integrate some LLM related code/logic in Function1and Function 2*
Simplest we can do is 1. pass some prompt with instructions to LLM with the User input. 2. The output of function 1 can be converted to Upper case and also we can pass the Length of Output.


In [24]:
def function1(input):
  complete_query = "Your task is to provide only topics based on the user query. Only output the topics among : [India, Cricket, FIFA]. If the asked query doesnot belong to given topic list then say I cannot identify the Topic. \
                    Don't include reasoning. Final resopnse to be under 250 Words only. Following is the user query : " + input
  response = llm.invoke(complete_query)
  return response.content

def function2(input): # this function will get response/output of function1
  upper_response = input.upper()

  length = len(input)
  final_response = f"Here is the topic in Upper Case : {upper_response}. \n Total length of this Topic is {length}"
  return final_response


In [25]:
llm_builder = Graph()

llm_builder.add_node("Agent", function1)
llm_builder.add_node("tool", function2)

llm_builder.add_edge("Agent", "tool")

llm_builder.set_entry_point("Agent")
llm_builder.set_finish_point("tool")

app = llm_builder.compile()




In [26]:
user_query = "Tell me about India"

response = app.invoke(user_query)
print(response)

Here is the topic in Upper Case : INDIA. 
 Total length of this Topic is 5


In [27]:
user_query = "Tell me about USA elections"

response = app.invoke(user_query)
print(response)

Here is the topic in Upper Case : I CANNOT IDENTIFY THE TOPIC.. 
 Total length of this Topic is 28


In [28]:
# Lets change the prompt of Agent a bit

def function1(input):
  complete_query = "Your task is to provide information in less than 150 Word. Only output for the topics among : [India, Cricket, USA]. If the asked query does not belong to given topic list then say Sorry, I Don't have Information... \
                    Don't include reasoning. Following is the user query : " + input
  response = llm.invoke(complete_query)
  return response.content

def function2(input): # this function will get response/output of function1
  upper_response = input.upper()

  length = len(input)
  final_response = f"Here is the topic in Upper Case : {upper_response}. \n Total length of this Topic is {length}"
  return final_response

In [29]:
llm_builder = Graph()

llm_builder.add_node("Agent", function1)
llm_builder.add_node("tool", function2)

llm_builder.add_edge("Agent", "tool")

llm_builder.set_entry_point("Agent")
llm_builder.set_finish_point("tool")

app = llm_builder.compile()


In [30]:
user_query = "Tell me about India"

response = app.invoke(user_query)
print(response)

Here is the topic in Upper Case : INDIA IS A COUNTRY LOCATED IN SOUTH ASIA. IT IS KNOWN FOR ITS DIVERSE CULTURE, RICH HISTORY, AND VIBRANT TRADITIONS. WITH A POPULATION OF OVER 1.3 BILLION PEOPLE, INDIA IS THE SECOND MOST POPULOUS COUNTRY IN THE WORLD. IT IS ALSO THE SEVENTH-LARGEST COUNTRY BY LAND AREA. INDIA IS A FEDERAL PARLIAMENTARY DEMOCRATIC REPUBLIC, WITH NEW DELHI AS ITS CAPITAL. THE OFFICIAL LANGUAGES ARE HINDI AND ENGLISH. THE ECONOMY OF INDIA IS ONE OF THE FASTEST-GROWING IN THE WORLD, WITH A STRONG FOCUS ON TECHNOLOGY, AGRICULTURE, AND SERVICES. INDIA IS ALSO FAMOUS FOR ITS CUISINE, INCLUDING DISHES SUCH AS CURRY, BIRYANI, AND DOSA. THE COUNTRY IS HOME TO ICONIC LANDMARKS SUCH AS THE TAJ MAHAL, JAIPUR'S AMBER FORT, AND MUMBAI'S GATEWAY OF INDIA.. 
 Total length of this Topic is 733


In [31]:
user_query = "Tell me about France"

response = app.invoke(user_query)
print(response)


Here is the topic in Upper Case : SORRY, I DON'T HAVE INFORMATION.... 
 Total length of this Topic is 34
